# Description du projet : 

Contexte :  L'INSEE est l'institut officiel français qui collecte des données de tous types sur le territoire français. Elles peuvent être démographiques (Naissances, Décès, Densité de la population...), économiques (Salaires, Entreprises par activité / taille...) et plus encore.
Ces données peuvent être d'une grande aide pour observer et mesurer les inégalités au sein de la population française.

## Objectif : Comparer les inégalités en France : 

* Entreprises en fonction de leur localisation, de leur taille. 
* Population en fonction du salaire et de la localisation.
* Focus sur une grande ville 

## Ressources à consulter : 
* Données : https://assets-datascientest.s3.eu-west-1.amazonaws.com/notebooks/power_bi/power_bi_datasets_projet/french_industry.zip


### Informations sur le nombre d'entreprises dans chaque ville française classées par taille.
<u>base_etablissement_par_tranche_effectif.csv :</u>

* CODGEO : ID géographique de la ville
* LIBGEO : nom de la ville
* REG : numéro de région
* DEP : numéro de département
* E14TST : nombre total d'entreprises dans la ville
* E14TS0ND : nombre d'entreprises de taille inconnue ou nulle dans la ville
* E14TS1 : nombre d'entreprises de 1 à 5 employés dans la ville
* E14TS6 : nombre d'entreprises de 6 à 9 employés dans la ville
* E14TS10 : nombre d'entreprises de 10 à 19 employés dans la ville
* E14TS20 : nombre d'entreprises de 20 à 49 employés dans la ville
* E14TS50 : nombre d'entreprises de 50 à 99 employés dans la ville
* E14TS100 :  nombre d'entreprises de 100 à 199 employés dans la ville
* E14TS200 : nombre d'entreprises de 200 à 499 employés dans la ville
* E14TS500 : nombre d'entreprises de plus de 500 employés dans la ville

In [1]:
# Suppression des warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
# Lecture fichier csv
df1 = pd.read_csv('base_etablissement_par_tranche_effectif.csv')
# Tous les nom de ville mis à la première lettre en masjuscule
df1.LIBGEO = df1.LIBGEO.str.title()
# Suppression des accents les plus communs
df1['LIBGEO'] = df1['LIBGEO'].str.replace("É","E")
df1['LIBGEO'] = df1['LIBGEO'].str.replace("Î","I")
#correction code geo erroné commune "L'Oudon"
df_temp = df1.loc[df1.CODGEO=='14697']
df_temp.CODGEO = '14472'
df1.loc[df1.CODGEO=='14697']=df_temp
# Code région stransformé en string
df1.REG = df1.REG.astype(str)
# Extraction des données GEO de la DF
df1_base = df1.iloc[:,0:4]
# Affichage des infos du dataset
df1_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36681 entries, 0 to 36680
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CODGEO  36681 non-null  object
 1   LIBGEO  36681 non-null  object
 2   REG     36681 non-null  object
 3   DEP     36681 non-null  object
dtypes: object(4)
memory usage: 1.1+ MB


### Données géographiques sur les villes françaises (principalement la latitude et la longitude, mais aussi les codes et les noms des régions/départements).
<u>name_geographic_information.csv : </u>

* code_région
* nom_région
* chef.lieu_région
* numéro_département
* nom_département
* préfecture
* numéro_circonscription
* nom_commune
* codes_postaux
* code_insee
* latitude
* longitude
* éloignement

In [3]:
# Lecture du fichier csv
df2 = pd.read_csv('name_geographic_information.csv')

# Correction du format 
df2.code_région = df2.code_région.astype(str)
df2.numéro_circonscription = df2.numéro_circonscription.astype(str)

# Ajout du '0' à code_insee
df2.code_insee = df2.code_insee.astype(str)
code_insee = df2.code_insee.values.tolist()
dep = df2.numéro_département.values.tolist()
code_insee_new = []
for i,j in zip(dep,code_insee):
    if i[0]=='0':
        j = '0' + str(j)
    code_insee_new.append(j)
df2.code_insee = code_insee_new

# Correction des valeurs erronée de la longitude
df2.longitude = df2.longitude.str.replace(",",".")
df2.longitude = df2.longitude.str.replace("-","0")
df2.longitude = df2.longitude.astype('float64')

# correction coordonnées par suppression des nan
display(df2.loc[33872:33873,'nom_département':])
df2.loc[33873:33873,'latitude':'éloignement'] = tuple(df2.loc[33872:33872,'latitude':'éloignement'].values)

# Majuscule sur premère lettres des noms de communes
df2.nom_commune = df2.nom_commune.str.title()

# Correction du code région de Mayotte
df_temp = df2.loc[df2.numéro_département=='976']
df_temp.code_région = '6'
df2.loc[df2.numéro_département=='976']=df_temp

# Correction du code région de Saint Pierre et Miquelon
df_temp = df2.loc[df2.numéro_département=='975']
df_temp.code_région = '5'
df2.loc[df2.numéro_département=='975']=df_temp

# Correction du nom de commune erroné
df_temp=df2.loc[df2.nom_commune=='Bandraboua']
df_temp.nom_commune='Saint-Pierre'
df_temp=df2.loc[df2.nom_commune=='Bandraboua'] = df_temp

#correction code geo erroné commune "L'Oudon"
df_temp = df2.loc[df2.code_insee=='14697']
df_temp.code_insee = '14472'
df2.loc[df2.code_insee=='14697']=df_temp

df2.info()

,nom_département,préfecture,numéro_circonscription,nom_commune,codes_postaux,code_insee,latitude,longitude,éloignement
33872,Tarn-et-Garonne,Montauban,1,Laguépie,82250,82088,44.141667,1.966667,-1.0
33873,Tarn-et-Garonne,Montauban,1,Laguépie,82250,82088,NaN,1.970000,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36840 entries, 0 to 36839
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   EU_circo                36840 non-null  object 
 1   code_région             36840 non-null  object 
 2   nom_région              36840 non-null  object 
 3   chef.lieu_région        36840 non-null  object 
 4   numéro_département      36840 non-null  object 
 5   nom_département         36840 non-null  object 
 6   préfecture              36840 non-null  object 
 7   numéro_circonscription  36840 non-null  object 
 8   nom_commune             36840 non-null  object 
 9   codes_postaux           36840 non-null  object 
 10  code_insee              36840 non-null  object 
 11  latitude                33912 non-null  float64
 12  longitude               33999 non-null  float64
 13  éloignement             33879 non-null  float64
dtypes: float64(3), object(11)
memory usage

In [4]:
# Suppression des colonnes non indispensables et qui compliquent la gestion des doublons
df2_base = df2.drop(labels=['numéro_circonscription','codes_postaux'],axis='columns')
# Suppression des doublons
df2_base = df2_base.drop_duplicates(keep='first')
# Vérification de suppression des doublons
df2_base.loc[df2_base.duplicated()]

,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,nom_commune,code_insee,latitude,longitude,éloignement


In [5]:
# Fusion des DataFrame "Etablissement par tranche d'effectifs" et "données géographiques"
all_content = pd.merge(df1_base,df2_base,how='outer',left_on=['LIBGEO','REG','DEP'],right_on=['nom_commune','code_région','numéro_département'])

In [6]:
# Taille de la DataFrame
all_content.shape

(37072, 16)

In [7]:
# Recherche des duplications
all_content_dupli = all_content.loc[all_content.duplicated(subset=['CODGEO','nom_commune'])]
# Affichage des duplications
all_content_dupli

,CODGEO,LIBGEO,REG,DEP,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,nom_commune,code_insee,latitude,longitude,éloignement
37044,NaN,NaN,NaN,NaN,Sud-Ouest,73,Midi-Pyrénées,Toulouse,81,Tarn,Albi,Saint-Sulpice,81271,NaN,NaN,NaN
37068,NaN,NaN,NaN,NaN,Outre-Mer,1,Guadeloupe,Basse-Terre,971,Guadeloupe,Basse-Terre,Saint-Martin,97127,NaN,NaN,NaN


In [8]:
# Extraction des départements de la liste des dupliqués
departements = all_content_dupli.loc[all_content_dupli.REG.isna()].numéro_département.unique()
# Affichage des départements
departements

array(['81', '971'], dtype=object)

In [9]:
# Création d'une dataframe
df_dep=pd.DataFrame(columns=['REG','DEP','EU_circo','code_région','nom_région','chef.lieu_région',
                             'nom_département','préfecture'])

# Recherche des départements dans la DF all_content
for dep in departements:
    # Extraction des lignes contenant le numero de département
    df_temp = all_content.loc[all_content.numéro_département==dep]
    # On selectionne la première ligne
    df_temp = df_temp.iloc[0]
    # Ajout à la DF créee
    df_dep = df_dep.append(df_temp[['REG','DEP','EU_circo','code_région','nom_région','chef.lieu_région',
                             'nom_département','préfecture']])

# Affichage des lignes contennant les infos des numéros de département concernés
df_dep.head()

,REG,DEP,EU_circo,code_région,nom_région,chef.lieu_région,nom_département,préfecture
33391,73,81,Sud-Ouest,73,Midi-Pyrénées,Toulouse,Tarn,Albi
36552,1,971,Outre-Mer,1,Guadeloupe,Basse-Terre,Guadeloupe,Basse-Terre


In [10]:
# Création d'une DF temporaire
temp_new = pd.DataFrame()
# on parcourt la DF obtenue precedemment contenant les infos par département
for i in range(len(df_dep)):
    # extraction des lignes dupliquées contenant le numéro de département
    temp = all_content_dupli.loc[all_content_dupli.numéro_département==df_dep.iloc[i,1]]
    # Remplissage des lignes avec les infos manquantes du département
    temp[['REG','DEP','EU_circo','code_région','nom_région','chef.lieu_région','nom_département','préfecture']] = df_dep.iloc[i]
    # Ajout à la DF temporaire
    temp_new = temp_new.append(temp)
# Affichage de la DF contenant les infos à compléter
temp_new.head()

,CODGEO,LIBGEO,REG,DEP,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,nom_commune,code_insee,latitude,longitude,éloignement
37044,NaN,NaN,73,81,Sud-Ouest,73,Midi-Pyrénées,Toulouse,81,Tarn,Albi,Saint-Sulpice,81271,NaN,NaN,NaN
37068,NaN,NaN,1,971,Outre-Mer,1,Guadeloupe,Basse-Terre,971,Guadeloupe,Basse-Terre,Saint-Martin,97127,NaN,NaN,NaN


In [11]:
# On complete la DF all_content avec les infos precedemment extraites
all_content[all_content.columns[2:11]] = all_content[all_content.columns[2:11]].fillna(value=temp_new[temp_new.columns[2:11]])
# On complète LIBGEO nan avec les noms de communes
all_content.LIBGEO = all_content.LIBGEO.fillna(value=all_content.nom_commune)
# On complète les noms de communes nan par LIBGEO
all_content.nom_commune = all_content.nom_commune.fillna(value=all_content.LIBGEO)
# On complete les code region nan par REG
all_content.code_région = all_content.code_région.fillna(value=all_content.REG)
# On complete la donnée REG nan par le code région
all_content.REG = all_content.REG.fillna(value=all_content.code_région)
# On complète les numéros de département nan par la donnée DEP
all_content.numéro_département = all_content.numéro_département.fillna(value=all_content.DEP)
# On complète la donnée DEP nan par le numero de departement
all_content.DEP = all_content.DEP.fillna(value=all_content.numéro_département)

# CodeGEO nan remplacés par code_insee
all_content.CODGEO = all_content.CODGEO.fillna(value=all_content.code_insee)
# On egalise le code insee avec le CODGEO
all_content.code_insee = all_content.CODGEO

# Changement de nom de 'Arbignieu' vers 'Arboys En Bugey'
df_temp = all_content.loc[all_content.CODGEO=='01015']
df_temp.LIBGEO = 'Arboys En Bugey'
all_content.loc[all_content.CODGEO=='01015']=df_temp

# On vérifie la disparition des lignes dupliquées
all_content_dupli = all_content.loc[all_content.duplicated(subset=['CODGEO','nom_commune'])]
# Vérification par affichage des duplications
all_content_dupli

,CODGEO,LIBGEO,REG,DEP,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,nom_commune,code_insee,latitude,longitude,éloignement


In [12]:
# On s'occuppe à présent des nan
all_content.isna().sum()

CODGEO                   0
LIBGEO                   0
REG                      0
DEP                      0
EU_circo               379
code_région              0
nom_région             379
chef.lieu_région       379
numéro_département       0
nom_département        379
préfecture             379
nom_commune              0
code_insee               0
latitude              3302
longitude             3217
éloignement           3305
dtype: int64

In [13]:
# extraction des numéros de département contenant des nan EU_circo
departements = all_content.loc[all_content.EU_circo.isna()].DEP.unique()
departements

array(['01', '02', '04', '05', '06', '07', '08', '10', '11', '12', '14',
       '15', '16', '17', '19', '21', '22', '23', '24', '26', '27', '28',
       '29', '2A', '30', '31', '33', '34', '35', '36', '37', '38', '39',
       '40', '41', '42', '43', '45', '47', '48', '49', '50', '51', '52',
       '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63',
       '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74',
       '76', '78', '79', '80', '81', '83', '85', '86', '88', '89', '90',
       '91', '94', '95', '976'], dtype=object)

In [14]:
# Création d'und DF temporaire
df_dep=pd.DataFrame(columns=['REG','DEP','EU_circo','code_région','nom_région','chef.lieu_région',
                              'nom_département','préfecture'])

# Recherche des département dans all content pour completer les nan
for dep in departements:
    # Extraction des lignes des départements concernés
    df_temp = all_content.loc[all_content.DEP==dep]
    # Extraction de la première ligne
    df_temp = df_temp.iloc[0]
    # Vérification de présence de nan dans la ligne extraite
    if df_temp[['REG','DEP','EU_circo','code_région','nom_région','chef.lieu_région',
                             'nom_département','préfecture']].isna().sum()>0:
        # Si presence de nan 
        # Extraction des lignes du département concerné (ici pour mayotte uniquement)
        df_temp_may = all_content.loc[all_content.DEP==dep]
        # Extraction des lignes sans nan
        df_temp = df_temp_may.loc[df_temp_may.EU_circo.notna()]
        # Extraction de la première ligne
        df_temp = df_temp.iloc[0]
    # Ajout à la DF temporaire
    df_dep = df_dep.append(df_temp[['REG','DEP','EU_circo','code_région','nom_région','chef.lieu_région',
                             'nom_département','préfecture']])

# Affichage de la DF contenant les infos pour compléter les nan
df_dep

,REG,DEP,EU_circo,code_région,nom_région,chef.lieu_région,nom_département,préfecture
0,82,01,Sud-Est,82,Rhône-Alpes,Lyon,Ain,Bourg-en-Bresse
419,22,02,Nord-Ouest,22,Picardie,Amiens,Aisne,Laon
1555,93,04,Sud-Est,93,Provence-Alpes-Côte d'Azur,Marseille,Alpes-de-Haute-Provence,Digne-les-Bains
1755,93,05,Sud-Est,93,Provence-Alpes-Côte d'Azur,Marseille,Hautes-Alpes,Gap
1927,93,06,Sud-Est,93,Provence-Alpes-Côte d'Azur,Marseille,Alpes-Maritimes,Nice
...,...,...,...,...,...,...,...,...
35946,43,90,Est,43,Franche-Comté,Besançon,Territoire de Belfort,Belfort
36048,11,91,Île-de-France,11,Île-de-France,Paris,Essonne,Évry
36320,11,94,Île-de-France,11,Île-de-France,Paris,Val-de-Marne,Créteil
36367,11,95,Île-de-France,11,Île-de-France,Paris,Val-d'Oise,Cergy


In [15]:
# Création d'une nouvelle DF temporaire
temp_new = pd.DataFrame()
# Recherche dans la liste des département contenant des nan
for i in range(len(df_dep)):
    # Extraction des lignes contenent le numéro de département
    temp = all_content.loc[all_content.DEP==df_dep.iloc[i,1]]
    # Extraction des données à completer
    temp[['REG','DEP','EU_circo','code_région','nom_région','chef.lieu_région','nom_département','préfecture']] = df_dep.iloc[i]
    # Ajour à la DF temporaire
    temp_new = temp_new.append(temp)
# Affichage de la DF temporaire
temp_new.head(3)

,CODGEO,LIBGEO,REG,DEP,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,nom_commune,code_insee,latitude,longitude,éloignement
0,01001,L'Abergement-Clémenciat,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,L'Abergement-Clémenciat,01001,46.15,4.916667,1.46
1,01002,L'Abergement-De-Varey,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,L'Abergement-De-Varey,01002,46.00,5.416667,1.00
2,01004,Ambérieu-En-Bugey,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,Ambérieu-En-Bugey,01004,45.95,5.350000,0.80


In [16]:
# Remplissage des nan dans la DF all content
all_content[all_content.columns[2:11]] = all_content[all_content.columns[2:11]].fillna(value=temp_new[temp_new.columns[2:11]])

In [17]:
# Verification de la dispartition des nan
all_content.isna().sum()

CODGEO                   0
LIBGEO                   0
REG                      0
DEP                      0
EU_circo                 0
code_région              0
nom_région               0
chef.lieu_région         0
numéro_département       0
nom_département          0
préfecture               0
nom_commune              0
code_insee               0
latitude              3302
longitude             3217
éloignement           3305
dtype: int64

In [18]:
# Verification que le code insee et GEO sont identiques
for i in range(len(all_content)):
    if all_content.iloc[i].CODGEO != all_content.iloc[i].code_insee:
        print(i)
        print(all_content.iloc[i].CODGEO)
        print(all_content.iloc[i].code_insee)

In [19]:
# Affichage de la dimension de la DF all_content
all_content.shape

(37072, 16)

In [20]:
# Recherche des duplication du CODGEO
all_content_dupli = all_content.loc[all_content.duplicated(subset=['CODGEO'])]
# Affichage des lignes dupliquées
all_content_dupli

,CODGEO,LIBGEO,REG,DEP,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,nom_commune,code_insee,latitude,longitude,éloignement
36681,01040,Béreyziat,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,Béreyziat,01040,46.366667,5.050000,1.71
36682,01170,Géovreissiat,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,Géovreissiat,01170,46.200000,5.566667,0.76
36683,02803,La Ville-Aux-Bois-Lès-Pontavert,22,02,Nord-Ouest,22,Picardie,Amiens,02,Aisne,Laon,La Ville-Aux-Bois-Lès-Pontavert,02803,NaN,NaN,NaN
36684,02676,Saint-Erme-Outre-Et-Ramecourt,22,02,Nord-Ouest,22,Picardie,Amiens,02,Aisne,Laon,Saint-Erme-Outre-Et-Ramecourt,02676,49.516667,3.850000,1.41
36685,02319,Fonsommes,22,02,Nord-Ouest,22,Picardie,Amiens,02,Aisne,Laon,Fonsommes,02319,49.900000,3.400000,1.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37062,91525,Roinville-Sous-Dourdan,11,91,Île-de-France,11,Île-de-France,Paris,91,Essonne,Évry,Roinville-Sous-Dourdan,91525,48.533333,2.033333,1.85
37063,91338,Limours En Hurepoix,11,91,Île-de-France,11,Île-de-France,Paris,91,Essonne,Évry,Limours En Hurepoix,91338,48.650000,2.083333,1.43
37064,91687,Viry-Chatillon,11,91,Île-de-France,11,Île-de-France,Paris,91,Essonne,Évry,Viry-Chatillon,91687,48.666667,2.383333,1.10
37065,94038,L'Hay-Les-Roses,11,94,Île-de-France,11,Île-de-France,Paris,94,Val-de-Marne,Créteil,L'Hay-Les-Roses,94038,48.783333,2.333333,1.92


In [21]:
# Suppression des lignes dupliquées pour les données GEO, 
# après observation de la DF pour les lignes CODGEO dupliquées on garde la dernière ligne pour conserver les infos GPS
all_content = all_content.drop_duplicates(subset=['CODGEO','REG','DEP','EU_circo','code_région','nom_région','chef.lieu_région','numéro_département','nom_département','préfecture','code_insee'],keep='last')

In [22]:
# Extraction des lignes code GEO dupliquées
all_content_dupli = all_content.loc[all_content.duplicated(subset=['CODGEO'])]
# Vérification de l'absence de duplication
all_content_dupli

,CODGEO,LIBGEO,REG,DEP,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,nom_commune,code_insee,latitude,longitude,éloignement


In [23]:
# Vérification des nan. Remarque le nbre de nan GPS à diminué par suppression des doublons
all_content.isna().sum()

CODGEO                   0
LIBGEO                   0
REG                      0
DEP                      0
EU_circo                 0
code_région              0
nom_région               0
chef.lieu_région         0
numéro_département       0
nom_département          0
préfecture               0
nom_commune              0
code_insee               0
latitude              2942
longitude             2857
éloignement           2945
dtype: int64

In [24]:
# Dimension de la DF all_content
all_content.shape

(36712, 16)

### Salaires par villes française par catégories d'emploi, âge et sexe

<u>net_salary_per_town_categories.csv :</u> 

* CODGEO : ID géographique de la ville
* LIBGEO : nom de la ville
* SNHM14 : salaire net moyen
* SNHMC14 : salaire net moyen par heure pour les cadres
* SNHMP14 : salaire net moyen par heure pour un cadre moyen
* SNHME14 : salaire net moyen par heure pour l'employé
* SNHMO14 :  salaire net moyen par heure pour le travailleur
* SNHMF14 : salaire net moyen pour les femmes
* SNHMFC14 : salaire net moyen par heure pour les cadres féminins
* SNHMFP14 : salaire net moyen par heure pour les cadres moyens féminins
* SNHMFE14 : salaire net moyen par heure pour une employée 
* SNHMFO14 : salaire net moyen par heure pour une travailleuse 
* SNHMH14 : salaire net moyen pour un homme
* SNHMHC14 : salaire net moyen par heure pour un cadre masculin
* SNHMHP14 : salaire net moyen par heure pour les cadres moyens masculins
* SNHMHE14 : salaire net moyen par heure pour un employé masculin
* SNHMHO14 : salaire net moyen par heure pour un travailleur masculin
* SNHM1814 : salaire net moyen par heure pour les 18-25 ans
* SNHM2614 : salaire net moyen par heure pour les 26-50 ans
* SNHM5014 : salaire net moyen par heure pour les >50 ans
* SNHMF1814 : salaire net moyen par heure pour les femmes âgées de 18 à 25 ans
* SNHMF2614 : salaire net moyen par heure pour les femmes âgées de 26 à 50 ans
* SNHMF5014 : salaire net moyen par heure pour les femmes de plus de 50 ans
* SNHMH1814 : salaire net moyen par heure pour les hommes âgés de 18 à 25 ans
* SNHMH2614 : salaire net moyen par heure pour les hommes âgés de 26 à 50 ans
* SNHMH5014 : salaire net moyen par heure pour les hommes de plus de 50 ans


In [25]:
# Lecture du fichier csv
df3 = pd.read_csv('net_salary_per_town_categories.csv')
# Mise en majuscule de la première lettre du nom de commune LIBGEO
df3.LIBGEO = df3.LIBGEO.str.title()
# Suppression des accents les plus courants sur la majuscule
df3['LIBGEO'] = df3['LIBGEO'].str.replace("É","E")
df3['LIBGEO'] = df3['LIBGEO'].str.replace("Î","I")
# Extraction des variables Geo uniquement
df3_base = df3.iloc[:,0:2]
# Affichage des premières de la DF
df3_base.head()

,CODGEO,LIBGEO
0,01004,Ambérieu-En-Bugey
1,01007,Ambronay
2,01014,Arbent
3,01024,Attignat
4,01025,Bâgé-La-Ville


In [26]:
# affiche de la dimension de la DF
df3_base.shape

(5136, 2)

In [27]:
# Recherche de nan
df3_base.isna().sum()

CODGEO    0
LIBGEO    0
dtype: int64

### Informations démographiques par ville, âge, sexe et mode de vie

<u>population.csv : </u>

* NIVGEO : geographic level (arrondissement, communes…)
* CODGEO : unique code for the town
* LIBGEO : name of the town
* MOCO : mode de cohabitation :
    * 11 = enfants vivant avec deux parents
    * 12 = enfants vivant avec un seul parent
    * 21 = adultes vivant en couple sans enfant
    * 22 = adultes vivant en couple avec enfants
    * 23 = adultes vivant seuls avec enfants
    * 31 = personnes étrangères à la famille vivant au foyer
    * 32 = personnes vivant seules
* AGE80_17 : catégorie d'âge (tranche de 5 ans) | ex : 0 -> personnes âgées de 0 à 4 ans
* SEXE : sexe, 1 pour homme | 2 pour femme
* NB : Nombre de personnes dans la catégorie

In [28]:
# Lecture du fichier de données population
df4 = pd.read_csv('population.csv',low_memory=False)
# Mise en majuscule de la première lettre du nom de commune LIBGEO 
df4.LIBGEO = df4.LIBGEO.str.title()
# Suppression des accents les plus courants sur la majuscule
df4['LIBGEO'] = df4['LIBGEO'].str.replace("É","E")
df4['LIBGEO'] = df4['LIBGEO'].str.replace("Î","I")
# Affichage des infos de la DF
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8536584 entries, 0 to 8536583
Data columns (total 7 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   NIVGEO     object
 1   CODGEO     object
 2   LIBGEO     object
 3   MOCO       int64 
 4   AGEQ80_17  int64 
 5   SEXE       int64 
 6   NB         int64 
dtypes: int64(4), object(3)
memory usage: 455.9+ MB


In [29]:
# On groupe les données par commune pour obtenir la population
# Création d'une nouvelle DF population
population = pd.DataFrame(df4.groupby(['NIVGEO','CODGEO','LIBGEO'])['NB'].sum(),index=None)
# Affichage de la dimension
population.shape

(35868, 1)

In [30]:
# Verification des nan
population.isna().sum()

NB    0
dtype: int64

In [31]:
# Fusion des DF salaires et population
df3_base_pop = pd.merge(population,df3_base,how='outer',on=['CODGEO','LIBGEO'])
# Affichage de la dimension de la DF
df3_base_pop.shape

(35984, 3)

In [32]:
# Vérification des nan
df3_base_pop.isna().sum()

CODGEO      0
LIBGEO      0
NB        116
dtype: int64

In [33]:
# Fusion des DF salaires/population et all_content (entreprises)
all_content_pop = pd.merge(all_content,df3_base_pop,how='outer',on=['CODGEO','LIBGEO'])
# Affichage de la dimension
all_content_pop.shape

(37242, 17)

In [35]:
# Affichage des premières lignes de la nouvelle DF
all_content_pop.head()

,CODGEO,LIBGEO,REG,DEP,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,nom_commune,code_insee,latitude,longitude,éloignement,NB
0,01001,L'Abergement-Clémenciat,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,L'Abergement-Clémenciat,01001,46.15,4.916667,1.46,765.0
1,01002,L'Abergement-De-Varey,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,L'Abergement-De-Varey,01002,46.00,5.416667,1.00,227.0
2,01004,Ambérieu-En-Bugey,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,Ambérieu-En-Bugey,01004,45.95,5.350000,0.80,13460.0
3,01005,Ambérieux-En-Dombes,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,Ambérieux-En-Dombes,01005,46.00,4.900000,1.72,1595.0
4,01006,Ambléon,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,Ambléon,01006,45.75,5.600000,2.01,84.0


In [36]:
# Affichage des nan
all_content_pop.isna().sum()

CODGEO                   0
LIBGEO                   0
REG                    530
DEP                    530
EU_circo               530
code_région            530
nom_région             530
chef.lieu_région       530
numéro_département     530
nom_département        530
préfecture             530
nom_commune            530
code_insee             530
latitude              3472
longitude             3387
éloignement           3475
NB                    1374
dtype: int64

In [37]:
# Suppression de la commune 'Chemillé-Melay' : doublon avec nom different
all_content_pop = all_content_pop.drop(labels=all_content_pop.loc[all_content_pop.LIBGEO=='Chemillé-Melay'].index,axis='index')

In [38]:
# Extraction des CODGEO dupliqués
all_content_pop_dupli = all_content_pop.loc[all_content_pop.duplicated(subset=['CODGEO'])]
# Affichage des lignes dupliquées
all_content_pop_dupli

,CODGEO,LIBGEO,REG,DEP,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,nom_commune,code_insee,latitude,longitude,éloignement,NB
36712,01040,Béréziat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,442.0
36713,01080,Champdor-Corcelles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,632.0
36714,01170,Béard-Géovreissiat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0
36715,01187,Haut Valromey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,717.0
36716,01204,Le Poizat-Lalleyriat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,695.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37236,91338,Limours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6635.0
37237,91525,Roinville,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1200.0
37238,91687,Viry-Châtillon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30975.0
37239,94038,L'Haÿ-Les-Roses,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29995.0


In [39]:
# Extraction de la population par commune NB
nb = all_content_pop_dupli.NB.values
# Affichage du nombre d'élement
len(nb)

528

In [40]:
# Après vérification de la structure des duplication de la DF, 
# on peut supprimer les lignes dupliquées en ne gardant que la première
all_content_pop = all_content_pop.drop_duplicates(subset=['CODGEO'],keep='first')

In [41]:
# Extraction des CODGEO des lignes dupliquées
codgeo = all_content_pop_dupli.CODGEO.values
# Creation d'une DF temporaire
df_codgeo=pd.DataFrame()
# Pore chaque codegeo dupliqué
for cod in codgeo:
    # extraction des lignes avec le code dupliqué
    df_temp = all_content_pop.loc[all_content_pop.CODGEO==cod]
    # Extraction de la première ligne
    df_temp = df_temp.iloc[0]
    # Ajout la la DF temporaire
    df_codgeo = df_codgeo.append(df_temp)
# On complete l'extrait avec la variable NB (population par commune)
df_codgeo.NB = nb
# Affichage de la DF
df_codgeo

,CODGEO,LIBGEO,REG,DEP,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,nom_commune,code_insee,latitude,longitude,éloignement,NB
36321,01040,Béreyziat,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,Béreyziat,01040,46.366667,5.050000,1.71,442.0
72,01080,Champdor,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,Champdor,01080,46.016667,5.600000,0.87,632.0
36322,01170,Géovreissiat,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,Géovreissiat,01170,46.200000,5.566667,0.76,1000.0
168,01187,Hotonnes,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,Hotonnes,01187,46.000000,5.683333,1.16,717.0
184,01204,Lalleyriat,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,Lalleyriat,01204,46.150000,5.716667,1.27,695.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36703,91338,Limours En Hurepoix,11,91,Île-de-France,11,Île-de-France,Paris,91,Essonne,Évry,Limours En Hurepoix,91338,48.650000,2.083333,1.43,6635.0
36702,91525,Roinville-Sous-Dourdan,11,91,Île-de-France,11,Île-de-France,Paris,91,Essonne,Évry,Roinville-Sous-Dourdan,91525,48.533333,2.033333,1.85,1200.0
36704,91687,Viry-Chatillon,11,91,Île-de-France,11,Île-de-France,Paris,91,Essonne,Évry,Viry-Chatillon,91687,48.666667,2.383333,1.10,30975.0
36705,94038,L'Hay-Les-Roses,11,94,Île-de-France,11,Île-de-France,Paris,94,Val-de-Marne,Créteil,L'Hay-Les-Roses,94038,48.783333,2.333333,1.92,29995.0


In [42]:
# Extraction des lignes dupliquées
all_content_pop_dupli = all_content_pop.loc[all_content_pop.duplicated(subset=['CODGEO'])]
# Affichage des lignes dupliquée
all_content_pop_dupli

,CODGEO,LIBGEO,REG,DEP,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,nom_commune,code_insee,latitude,longitude,éloignement,NB


In [43]:
# Affichage du nombre de nan par colonne
all_content_pop.isna().sum()

CODGEO                   0
LIBGEO                   0
REG                      1
DEP                      1
EU_circo                 1
code_région              1
nom_région               1
chef.lieu_région         1
numéro_département       1
nom_département          1
préfecture               1
nom_commune              1
code_insee               1
latitude              2943
longitude             2858
éloignement           2946
NB                    1373
dtype: int64

In [44]:
# Extraction de la ligne avec des nan
all_content_pop_nan = all_content_pop.loc[all_content_pop['EU_circo'].isna()]
# Affichage de la ligne
all_content_pop_nan

,CODGEO,LIBGEO,REG,DEP,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,nom_commune,code_insee,latitude,longitude,éloignement,NB
37073,55138,Culey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.0


In [45]:
# creation d'und DF temporaire
df_dep=pd.DataFrame(columns=['REG','DEP','EU_circo','code_région','nom_région','chef.lieu_région',
                             'nom_département','préfecture'])

# Extraction des lignes du département 
df_temp = all_content_pop.loc[all_content_pop.DEP=='55']
# Extraction de la première ligne
df_temp = df_temp.iloc[0]
# Extraction des colonnes contenant les infos manquantes
df_dep = df_dep.append(df_temp[['REG','DEP','EU_circo','code_région','nom_région','chef.lieu_région',
                             'nom_département','préfecture']])
# Affichage de la DF
df_dep

,REG,DEP,EU_circo,code_région,nom_région,chef.lieu_région,nom_département,préfecture
20996,41,55,Est,41,Lorraine,Metz,Meuse,Bar-le-Duc


In [46]:
# Creation d'une DF temporaire
temp_new = pd.DataFrame()
# Extraction de la ligne contenant des nan
temp = all_content_pop_nan.loc[all_content_pop_nan.CODGEO=='55138']
# On complete la ligne avec les infos extraites précedemment
temp[['REG','DEP','EU_circo','code_région','nom_région','chef.lieu_région','nom_département','préfecture']] = df_dep.iloc[0]
# Ajout à la DF temporaire
temp_new = temp_new.append(temp)
# On complete les données manquantes
temp_new.nom_commune = temp_new.LIBGEO
temp_new.code_insee = temp_new.CODGEO
temp_new.numéro_département = temp_new.DEP
# Affichage de la ligne completée
display(temp_new)
# On remplit les nan dans la DF principale
all_content_pop[all_content_pop.columns[2:13]] = all_content_pop[all_content_pop.columns[2:13]].fillna(value=temp_new[temp_new.columns[2:13]])

,CODGEO,LIBGEO,REG,DEP,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,nom_commune,code_insee,latitude,longitude,éloignement,NB
37073,55138,Culey,41,55,Est,41,Lorraine,Metz,55,Meuse,Bar-le-Duc,Culey,55138,NaN,NaN,NaN,144.0


In [47]:
# Affichage de la dimension de la DF
all_content_pop.shape

(36713, 17)

In [48]:
# Affichage des nan
all_content_pop.isna().sum()

CODGEO                   0
LIBGEO                   0
REG                      0
DEP                      0
EU_circo                 0
code_région              0
nom_région               0
chef.lieu_région         0
numéro_département       0
nom_département          0
préfecture               0
nom_commune              0
code_insee               0
latitude              2943
longitude             2858
éloignement           2946
NB                    1373
dtype: int64

In [49]:
# Affichage des nan restant : GPS et population par commune
all_content_pop.loc[all_content_pop.isna().any(axis=1)]

,CODGEO,LIBGEO,REG,DEP,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,nom_commune,code_insee,latitude,longitude,éloignement,NB
26,01030,Beauregard,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,Beauregard,01030,NaN,NaN,NaN,888.0
32,01036,Belmont-Luthézieu,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,Belmont-Luthézieu,01036,NaN,NaN,NaN,504.0
54,01061,Brens,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,Brens,01061,NaN,NaN,NaN,1165.0
56,01063,Brion,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,Brion,01063,NaN,NaN,NaN,466.0
72,01080,Champdor,82,01,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,Champdor,01080,46.016667,5.6,0.87,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36708,97127,Saint-Martin,1,971,Outre-Mer,1,Guadeloupe,Basse-Terre,971,Guadeloupe,Basse-Terre,Saint-Martin,97127,NaN,NaN,NaN,NaN
36709,97501,Acoua,5,975,Outre-Mer,5,Saint-Pierre-et-Miquelon,Saint-Pierre,975,Saint-Pierre-et-Miquelon,Saint-Pierre,Acoua,97501,NaN,NaN,NaN,NaN
36710,97502,Saint-Pierre,5,975,Outre-Mer,5,Saint-Pierre-et-Miquelon,Saint-Pierre,975,Saint-Pierre-et-Miquelon,Saint-Pierre,Saint-Pierre,97502,NaN,NaN,NaN,NaN
36711,97503,Bandrele,5,975,Outre-Mer,5,Saint-Pierre-et-Miquelon,Saint-Pierre,975,Saint-Pierre-et-Miquelon,Saint-Pierre,Bandrele,97503,NaN,NaN,NaN,NaN


In [57]:
# CODGEO : ID géographique de la ville
# LIBGEO : nom de la ville
# REG : numéro de région
# DEP : numéro de département
# E14TST : nombre total d'entreprises dans la ville
# E14TS0ND : nombre d'entreprises de taille inconnue ou nulle dans la ville
# E14TS1 : nombre d'entreprises de 1 à 5 employés dans la ville
# E14TS6 : nombre d'entreprises de 6 à 9 employés dans la ville
# E14TS10 : nombre d'entreprises de 10 à 19 employés dans la ville
# E14TS20 : nombre d'entreprises de 20 à 49 employés dans la ville
# E14TS50 : nombre d'entreprises de 50 à 99 employés dans la ville
# E14TS100 : nombre d'entreprises de 100 à 199 employés dans la ville
# E14TS200 : nombre d'entreprises de 200 à 499 employés dans la ville
# E14TS500 : nombre d'entreprises de plus de 500 employés dans la ville
df1_E14T = df1.drop(labels=['LIBGEO','REG','DEP'],axis='columns')
df1_E14T

,CODGEO,E14TST,E14TS0ND,E14TS1,E14TS6,E14TS10,E14TS20,E14TS50,E14TS100,E14TS200,E14TS500
0,01001,25,22,1,2,0,0,0,0,0,0
1,01002,10,9,1,0,0,0,0,0,0,0
2,01004,996,577,272,63,46,24,9,3,2,0
3,01005,99,73,20,3,1,2,0,0,0,0
4,01006,4,4,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
36676,97613,169,169,0,0,0,0,0,0,0,0
36677,97614,230,230,0,0,0,0,0,0,0,0
36678,97615,569,569,0,0,0,0,0,0,0,0
36679,97616,345,345,0,0,0,0,0,0,0,0


In [58]:
# Fusion de all content pop avec les données des entreprises 
all_content_pop_entr = pd.merge(all_content_pop,df1_E14T,how='left',on='CODGEO')
all_content_pop_entr.shape

(36713, 27)

In [59]:
all_content_pop_entr.isna().sum()

CODGEO                   0
LIBGEO                   0
REG                      0
DEP                      0
EU_circo                 0
code_région              0
nom_région               0
chef.lieu_région         0
numéro_département       0
nom_département          0
préfecture               0
nom_commune              0
code_insee               0
latitude              2943
longitude             2858
éloignement           2946
NB                    1373
E14TST                  32
E14TS0ND                32
E14TS1                  32
E14TS6                  32
E14TS10                 32
E14TS20                 32
E14TS50                 32
E14TS100                32
E14TS200                32
E14TS500                32
dtype: int64

In [61]:
# CODGEO : ID géographique de la ville
# LIBGEO : nom de la ville
# SNHM14 : salaire net moyen
# SNHMC14 : salaire net moyen par heure pour les cadres
# SNHMP14 : salaire net moyen par heure pour un cadre moyen
# SNHME14 : salaire net moyen par heure pour l'employé
# SNHMO14 : salaire net moyen par heure pour le travailleur
# SNHMF14 : salaire net moyen pour les femmes
# SNHMFC14 : salaire net moyen par heure pour les cadres féminins
# SNHMFP14 : salaire net moyen par heure pour les cadres moyens féminins
# SNHMFE14 : salaire net moyen par heure pour une employée
# SNHMFO14 : salaire net moyen par heure pour une travailleuse
# SNHMH14 : salaire net moyen pour un homme
# SNHMHC14 : salaire net moyen par heure pour un cadre masculin
# SNHMHP14 : salaire net moyen par heure pour les cadres moyens masculins
# SNHMHE14 : salaire net moyen par heure pour un employé masculin
# SNHMHO14 : salaire net moyen par heure pour un travailleur masculin
# SNHM1814 : salaire net moyen par heure pour les 18-25 ans
# SNHM2614 : salaire net moyen par heure pour les 26-50 ans
# SNHM5014 : salaire net moyen par heure pour les >50 ans
# SNHMF1814 : salaire net moyen par heure pour les femmes âgées de 18 à 25 ans
# SNHMF2614 : salaire net moyen par heure pour les femmes âgées de 26 à 50 ans
# SNHMF5014 : salaire net moyen par heure pour les femmes de plus de 50 ans
# SNHMH1814 : salaire net moyen par heure pour les hommes âgés de 18 à 25 ans
# SNHMH2614 : salaire net moyen par heure pour les hommes âgés de 26 à 50 ans
# SNHMH5014 : salaire net moyen par heure pour les hommes de plus de 50 ans
df3_SNHM14 = df3.drop(labels='LIBGEO',axis='columns')
df3_SNHM14

,CODGEO,SNHM14,SNHMC14,SNHMP14,SNHME14,SNHMO14,SNHMF14,SNHMFC14,SNHMFP14,SNHMFE14,...,SNHMHO14,SNHM1814,SNHM2614,SNHM5014,SNHMF1814,SNHMF2614,SNHMF5014,SNHMH1814,SNHMH2614,SNHMH5014
0,01004,13.7,24.2,15.5,10.3,11.2,11.6,19.1,13.2,10.1,...,11.6,10.5,13.7,16.1,9.7,11.8,12.5,11.0,14.9,18.6
1,01007,13.5,22.1,14.7,10.7,11.4,11.9,19.0,13.3,10.6,...,11.7,9.8,13.8,14.6,9.2,12.2,12.5,10.2,14.9,16.4
2,01014,13.5,27.6,15.6,11.1,11.1,10.9,19.5,11.7,10.8,...,11.8,9.3,13.3,16.0,8.9,10.6,12.5,9.6,15.1,18.6
3,01024,12.9,21.8,14.1,11.0,11.3,11.4,19.0,13.0,10.3,...,11.6,9.6,12.9,14.2,9.3,11.4,12.2,9.7,13.8,15.9
4,01025,13.0,22.8,14.1,10.5,11.1,11.6,19.4,13.6,10.2,...,11.4,9.4,12.8,15.2,9.0,11.8,12.3,9.7,13.4,16.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5131,97420,12.9,24.5,15.4,10.9,10.9,12.4,21.6,14.7,10.8,...,11.1,9.2,12.8,15.4,9.3,12.3,14.7,9.2,13.1,15.8
5132,97421,10.4,17.3,13.8,9.6,9.8,9.8,13.2,11.4,9.6,...,10.0,9.0,10.6,11.4,8.9,9.9,10.5,9.0,11.0,11.7
5133,97422,12.0,23.3,14.7,10.3,10.5,11.5,21.3,14.1,10.2,...,10.6,8.9,11.9,14.9,8.9,11.4,13.9,9.0,12.1,15.4
5134,97423,11.4,22.6,13.4,10.1,10.5,10.8,15.9,12.4,10.1,...,10.5,8.9,11.5,13.1,8.9,11.1,11.4,9.0,11.8,13.7


In [62]:
# Fusion de all content pop entreprises avec les données de salaire
all_content_2014 = pd.merge(all_content_pop_entr,df3_SNHM14,how='left',on='CODGEO')
all_content_2014.shape

(36713, 51)

In [63]:
all_content_2014.isna().sum()

CODGEO                    0
LIBGEO                    0
REG                       0
DEP                       0
EU_circo                  0
code_région               0
nom_région                0
chef.lieu_région          0
numéro_département        0
nom_département           0
préfecture                0
nom_commune               0
code_insee                0
latitude               2943
longitude              2858
éloignement            2946
NB                     1373
E14TST                   32
E14TS0ND                 32
E14TS1                   32
E14TS6                   32
E14TS10                  32
E14TS20                  32
E14TS50                  32
E14TS100                 32
E14TS200                 32
E14TS500                 32
SNHM14                31577
SNHMC14               31577
SNHMP14               31577
SNHME14               31577
SNHMO14               31577
SNHMF14               31577
SNHMFC14              31577
SNHMFP14              31577
SNHMFE14            

In [64]:
# Sauvegarde de la DF all_content_pop dans un fichier csv
all_content_2014.to_csv("all_content_2014.csv",encoding= "utf-8")

### Meta dossier complet INSEE

<u>meta_dossier_complet.csv</u>

* COD_VAR : code de la variable
* LIB_VAR : libellé de la variable
* LIB_VAR_LONG : libellé long de la variable
* COD_MOD : Modalité des prises par les variables
* LIB_MOD : Libellé des modalités
* TYPE_VAR : Type de variables
* LONG_VAR : longueur des variables
* THEME : Thème associé à la variable
* SOURCE : Source utilisée pour la variable

In [ ]:
df5 = pd.read_csv('meta_dossier_complet.csv',sep=';',low_memory=False)
df5.COD_VAR.unique()

In [ ]:
# theme_var = pd.DataFrame(theme_var.reset_index())
# theme_var.THEME.unique()

In [ ]:
df5.THEME.value_counts()

In [ ]:
theme_var=df5.groupby(['THEME','COD_VAR','LIB_VAR','LIB_VAR_LONG']).size().unstack().fillna(0)

nbre_ligne = theme_var.shape[0]
pd.set_option("display.max_rows",nbre_ligne)

theme_var

In [ ]:
caract_ets_columns = df5.loc[df5['THEME']=='Caractéristiques des établissements']['COD_VAR'].values
familles_pop_columns = df5.loc[df5['THEME']=='Couples - Familles - Ménages']['COD_VAR'].values
salaires_columns = df5.loc[df5['THEME']=="Salaires et revenus d'activité"]['COD_VAR'].values

### dossier complet INSEE

<u>dossier_complet.csv</u>


In [ ]:
df6 = pd.read_csv('dossier_complet.csv',sep=';',low_memory=False)
df6.shape

In [ ]:
df6_new = df6.copy()
all_content_pop_df6 = pd.merge(df6_new,all_content_pop,how='outer',on='CODGEO')
all_content_pop_df6  = all_content_pop_df6.sort_values('CODGEO')

In [ ]:
all_content_pop_df6.shape

In [ ]:
temp = all_content_pop_df6.isna().sum() 
temp

In [ ]:
temp[(temp>31500) & (temp<36700)]

In [ ]:
temp[(temp>3100) & (temp<31500)]

In [ ]:
temp[(temp>2000) & (temp<3100)]

In [ ]:
temp[(temp>1900) & (temp<2000)]

In [ ]:
temp[(temp>1500) & (temp<1900)]

In [ ]:
temp[(temp>100) & (temp<1500)]

In [ ]:
temp[temp<100]

In [ ]:
code_insee_nan = all_content_pop_df6.loc[all_content_pop_df6.code_insee.isna()]
code_insee_nan

In [ ]:
all_content_pop_df6.loc[all_content_pop_df6.duplicated(subset='CODGEO')]

In [ ]:
code_insee = code_insee_nan.CODGEO.values
code_insee

In [ ]:
all_content_pop_df6.loc[all_content_pop_df6.DEP=='75']

In [ ]:
'75056' in df6.CODGEO.values

# Construction salaire/localisation 2020

## Salaires et revenus d'activité

* SNHM1820	: Salaire net horaire moyen 18 à 25 ans en 2020 (€)
* SNHM20	: Salaire net horaire moyen en 2020 (€)
* SNHM2620	: Salaire net horaire moyen 26 à 50 ans en 2020 (€)
* SNHM5020	: Salaire net horaire moyen plus de 50 ans en 2020 (€)
* SNHMC20	: Salaire net hor. Moy. Cadres sup. en 2020 (€)
* SNHME20	: Salaire net hor. moy. employés en 2020 (€)
* SNHMF1820	: Salaire net horaire moyen F 18 à 25 ans en 2020 (€)
* SNHMF20	: Salaire net horaire moyen F en 2020 (€)	
* SNHMF2620	: Salaire net horaire moyen F 26 à 50 ans en 2020 (€)
* SNHMF5020	: Salaire net horaire moyen F plus de 50 ans en 2020 (€)
* SNHMFC20	: Salaire net hor. moy. F cadres sup. en 2020 (€)
* SNHMFE20	: Salaire net hor. moy. F employés en 2020 (€)
* SNHMFO20	: Salaire net hor. moy. F ouvriers en 2020 (€)
* SNHMFP20	: Salaire net hor. moy. F prof inter. en 2020 (€)	
* SNHMH1820	: Salaire net horaire moyen H 18 à 25 ans en 2020 (€)	
* SNHMH20	: Salaire net horaire moyen H en 2020 (€)	
* SNHMH2620	: Salaire net horaire moyen H 26 à 50 ans en 2020 (€)
* SNHMH5020	: Salaire net horaire moyen H plus de 50 ans en 2020 (€)
* SNHMHC20	: Salaire net hor. moy. H cadres sup. en 2020 (€)	
* SNHMHE20	: Salaire net hor. moy. H employés en 2020 (€)	
* SNHMHO20	: Salaire net hor. moy. H ouvriers en 2020 (€)	
* SNHMHP20	: Salaire net hor. moy. H prof inter. en 2020 (€)	
* SNHMO20	: Salaire net hor. moy. ouvriers en 2020 (€)	
* SNHMP20	: Salaire net hor. moy. prof inter. en 2020 (€)	

In [ ]:
all_content_pop_columns = all_content_pop.columns.tolist()
salaires_columns = ['P19_POP','SUPERF'] + df5.loc[df5['THEME']=="Salaires et revenus d'activité"]['COD_VAR'].values.tolist()
salaires_columns = all_content_pop_columns[0:3] + salaires_columns + all_content_pop_columns[3:-1]
all_content_pop_df6_salaires = all_content_pop_df6[salaires_columns]
all_content_pop_df6_salaires = all_content_pop_df6_salaires.loc[all_content_pop_df6_salaires['SNHM20'].notna()]

In [ ]:
all_content_pop_df6_salaires.shape

In [ ]:
all_content_pop_df6_salaires.isna().sum()

In [ ]:
all_content_pop_df6_salaires

# Construction localisation / taille entreprises 2020

## Caractéristiques des établissements	

* **ETPTEF120**	     : Postes des Ets actifs de 1 à 9 salariés fin 2020
* **ETPTEF1020**	 : Postes des Ets actifs de 10 à 19 salariés fin 2020
* **ETPTEF2020**	 : Postes des Ets actifs de 20 à 49 salariés fin 2020
* **ETPTEF5020**	 : Postes des Ets actifs de 50 à 99 salariés fin 2020
* **ETPTEFCP20**	 : Postes des Ets actifs de 100 salariés ou plus fin 2020

* **ETPTOT20**	 : Postes des Ets actifs fin 2020
* **ETTEF020**	 : Ets actifs sans salarié fin 2020
* **ETTEF120**	 : Ets actifs de 1 à 9 salariés fin 2020
* **ETTEF1020**	 : Ets actifs de 10 à 19 salariés fin 2020	
* **ETTEF2020**	 : Ets actifs de 20 à 49 salariés fin 2020
* **ETTEF5020**	 : Ets actifs de 50 salariés ou plus fin 2020	
* **ETTOT20**	 : Total Ets actifs 

* ETASSMAT20 : Particuliers employeurs assistants mat fin 2020	
* ETAUTRES20 : Particuliers employeurs autres fin 2020	

* ETAZ020    : Ets actifs agriculture sans salarié fin 2020
* ETAZ1020   : Ets actifs agriculture 10 à 19 sal fin 2020	
* ETAZ120    : Ets actifs agriculture 1 à 9 salariés fin 2020	
* ETAZ20     : Ets actifs agriculture fin 2020	
* ETAZ2020   : Ets actifs agriculture 20 à 49 sal fin 2020	
* ETAZ5020   : Ets actifs agriculture 50 sal ou plus fin 2020

* ETBE020	 : Ets actifs industrie sans salarié fin 2020
* ETBE1020	 : Ets actifs industrie 10 à 19 sal fin 2020
* ETBE120	 : Ets actifs industrie 1 à 9 salariés fin 2020
* ETBE20	 : Ets actifs industrie fin 2020
* ETBE2020	 : Ets actifs industrie 20 à 49 sal fin 2020
* ETBE5020	 : Ets actifs industrie 50 sal ou plus fin 2020

* ETFZ020	 : Ets actifs construction sans salarié fin 2020
* ETFZ1020	 : Ets actifs construction 10 à 19 sal fin 2020
* ETFZ120	 : Ets actifs construction 1 à 9 sal fin 2020
* ETFZ20	 : Ets actifs construction fin 2020
* ETFZ2020	 : Ets actifs construction 20 à 49 sal fin 2020
* ETFZ5020	 : Ets actifs construction 50 sal ou + fin 2020

* ETGU020	 : Ets actifs commerce et services marchands sans salarié fin 2020
* ETGU1020	 : Ets actifs commerce et services marchands 10 à 19 salariés fin 2020	
* ETGU120	 : Ets actifs commerce et services marchands 1 à 9 salariés fin 2020
* ETGU20	 : Ets actifs commerce et services marchands fin 2020
* ETGU2020	 : Ets actifs commerce et services marchands 20 à 49 salariés fin 2020	
* ETGU5020	 : Ets actifs commerce et services marchands 50 sal ou plus fin 2020

* ETGZ020	 : dont Ets actifs commerce sans salarié fin 2020
* ETGZ1020	 : dont Ets actifs commerce 10 à 19 salariés fin 2020
* ETGZ120	 : dont Ets actifs commerce 1 à 9 salariés fin 2020
* ETGZ20	 : dont Ets actifs commerce fin 2020
* ETGZ2020	 : dont Ets actifs commerce 20 à 49 salariés fin 2020
* ETGZ5020	 : dont Ets actifs commerce 50 sal ou plus fin 2020

* ETNPRES20	 : Ets actifs sphère productive fin 2020
* ETNPRESPUB20	 : Ets actifs sphère productive public fin 2020

* ETOQ020	 : Ets actifs services non marchands sans salarié fin 2020
* ETOQ1020	 : Ets actifs services non marchands 10 à 19 salariés fin 2020	
* ETOQ120	 : Ets actifs services non marchands 1 à 9 salariés fin 2020
* ETOQ20	 : Ets actifs services non marchands fin 2020
* ETOQ2020	 : Ets actifs services non marchands 20 à 49 salariés fin 2020
* ETOQ5020	 : Ets actifs services non marchands 50 sal ou plus fin 2020

* ETPAZ1020	 : Postes des Ets actifs agriculture 10 à 19 salariés fin 2020
* ETPAZ120	 : Postes des Ets actifs agriculture 1 à 9 salariés fin 2020
* ETPAZ20	 : Postes des Ets actifs agriculture fin 2020
* ETPAZ2020	 : Postes des Ets actifs agriculture 20 à 49 salariés fin 2020
* ETPAZ5020	 : Postes des Ets actifs agriculture 50 à 99 salariés fin 2020
* ETPAZCP20	 : Postes des Ets actifs agriculture 100 sal ou + fin 2020

* ETPBE1020	 : Postes des Ets actifs industrie 10 à 19 sal fin 2020
* ETPBE120	 : Postes des Ets actifs industrie 1 à 9 salariés fin 2020
* ETPBE20	 : Postes des Ets actifs de l'industrie fin 2020
* ETPBE2020	 : Postes des Ets actifs industrie 20 à 49 sal fin 2020
* ETPBE5020	 : Postes des Ets actifs industrie 50 à 99 sal fin 2020
* ETPBECP20	 : Postes des Ets actifs industrie 100 sal ou + fin 2020

* ETPFZ1020	 : Postes des Ets actifs construction 10 à 19 sal fin 2020
* ETPFZ120	 : Postes des Ets actifs construction 1 à 9 sal fin 2020
* ETPFZ20	 : Postes des Ets actifs de la construction fin 2020
* ETPFZ2020	 : Postes des Ets actifs construction 20 à 49 sal fin 2020
* ETPFZ5020	 : Postes des Ets actifs construction 50 à 99 sal fin 2020
* ETPFZCP20	 : Postes des Ets actifs construction 100 sal ou + fin 2020

* ETPGU1020	 : Postes des Ets actifs commerce et services marchands 10 à 19 salariés fin 2020
* ETPGU120	 : Postes des Ets actifs commerce et services marchands 1 à 9 sal fin 2020
* ETPGU20	 : Postes des Ets actifs du commerce et services marchands fin 2020
* ETPGU2020	 : Postes des Ets actifs commerce et services marchands 20 à 49 salariés fin 2020
* ETPGU5020	 : Postes des Ets actifs commerce et services marchands 50 à 99 sal fin 2020
* ETPGUCP20	 : Postes des Ets actifs commerce et services marchands 100 sal ou + fin 2020

* ETPGZ1020	 : dont Postes des Ets actifs commerce 10 à 19 salariés fin 2020
* ETPGZ120	 : dont Postes des Ets actifs commerce 1 à 9 salariés fin 2020
* ETPGZ20	 : dont Postes des Ets actifs du commerce fin 2020
* ETPGZ2020	 : dont Postes des Ets actifs commerce 20 à 49 salariés fin 2020
* ETPGZ5020	 : dont Postes des Ets actifs commerce 50 à 99 sal fin 2020
* ETPGZCP20	 : dont Postes des Ets actifs commerce 100 sal ou + fin 2020

* ETPNPRES20	 : Postes Ets actifs sphère productive fin 2020
* ETPNPRESPUB20	 : Postes Ets actifs sphère productive public fin 2020

* ETPOQ1020	 : Postes des Ets actifs services non marchands 10 à 19 sal fin 2020
* ETPOQ120	 : Postes des Ets actifs services non marchands 1 à 9 salariés fin 2020
* ETPOQ20	 : Postes des Ets actifs services non marchands fin 2020
* ETPOQ2020	 : Postes des Ets actifs services non marchands 20 à 49 sal fin 2020
* ETPOQ5020	 : Postes des Ets actifs services non marchands 50 à 99 sal fin 2020
* ETPOQCP20	 : Postes des Ets actifs services non marchands 100 sal ou + fin 2020

* ETPPRES20	 : Postes Ets actifs sphère présentielle fin 2020
* ETPPRESPUB20	 : Postes Ets actifs sphère présentielle public fin 2020
* ETPRES20	 : Ets actifs sphère présentielle fin 2020
* ETPRESPUB20	 : Ets actifs sphère présentielle public fin 2020


In [ ]:
caract_ets_columns = ['P19_POP','SUPERF'] + df5.loc[df5['THEME']=='Caractéristiques des établissements']['COD_VAR'].values.tolist()
caract_ets_columns = all_content_pop_columns[0:3] + caract_ets_columns + all_content_pop_columns[3:-1]
all_content_pop_df6_entreprises = all_content_pop_df6[caract_ets_columns]
all_content_pop_df6_entreprises = all_content_pop_df6_entreprises.loc[all_content_pop_df6_entreprises['ETTOT20'].notna()]

In [ ]:
all_content_pop_df6_entreprises.shape

In [ ]:
all_content_pop_df6_entreprises.isna().sum()

In [ ]:
all_content_pop_df6_entreprises

In [ ]:
code_insee_entr_nan = all_content_pop_df6_entreprises.loc[all_content_pop_df6_entreprises.code_insee.isna()]
code_insee_entr_nan